# 씨마켓플레이스

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
from matplotlib import font_manager as fm

# 폰트 경로 직접 지정
font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'  # Mac 기본 폰트
font_prop = fm.FontProperties(fname=font_path)
# (, fontproperties=font_prop) 직접 추가해야 한글화 
# rcParams에 적용
rcParams['font.family'] = font_prop.get_name()
rcParams['axes.unicode_minus'] = False



# 데이터 로드

In [61]:
df = pd.read_csv('/Users/c-market/Downloads/data_list.csv', low_memory=False)

df.columns = df.columns.str.replace(' ', '_')

In [55]:
df

,NO,ROWNUM,totalCount,pdt_cd,pdt_share,pdt_share_txt,send_chk,send_chk_txt,pdt_code,pdt_name,...,pdt_cas,pdt_price,pdt_request_date,pdt_simg_type,pdt_simg,pdt_simg_org,pdt_simg_size,brand_name,dam_name,pdt_simg_file_key
0,1,104488,104488,100001,Y,공유,Y,전송완료,SO,전산기록지(80컬럼/양미싱),...,NaN,25600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
1,2,104487,104488,100002,Y,공유,Y,전송완료,SO,전산기록지(132컬럼/양미싱),...,NaN,36100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
2,3,104486,104488,100003,Y,공유,Y,전송완료,SO,팩스용지(15mX210mm/한솔),...,NaN,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
3,4,104485,104488,100004,Y,공유,Y,전송완료,SO,팩스용지(15mX216mm/한솔),...,NaN,2100,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
4,5,104484,104488,100005,Y,공유,Y,전송완료,SO,팩스용지(30mX210mm/한솔),...,NaN,3600,50:34.9,U,NaN,NaN,NaN,오피스디포,윤갑석,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104483,104484,5,104488,204575,Y,공유,Y,전송완료,SZ,BKA-001 무릎 보호대 K,...,NaN,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN
104484,104485,4,104488,204574,Y,공유,Y,전송완료,SZ,BKH-002 손목 보호대,...,NaN,0,25:45.8,NaN,cmarket_empty.png,NaN,NaN,NaN,백제현,NaN
104485,104486,3,104488,204585,Y,공유,Y,전송완료,SZ,[고려에이스] Centrifuge Tube Rack 스텐원심관랙,...,NaN,0,31:18.6,NaN,cmarket_empty.png,NaN,NaN,고려에이스,박예린,NaN
104486,104487,2,104488,204587,Y,공유,Y,전송완료,SZ,Bambu 저온용 재사용 가능 스풀,...,NaN,0,52:19.6,NaN,cmarket_empty.png,NaN,NaN,뱀부랩,안재용,NaN


In [56]:
def summary(df):
    summry = pd.DataFrame(df.dtypes, columns=['data type'])
    summry['#missing'] = df.isnull().sum().values
    summry['Duplicate'] = df.duplicated().sum()
    summry['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summry['min'] = desc['min'].values
    summry['max'] = desc['max'].values
    summry['avg'] = desc['mean'].values
    summry['std dev'] = desc['std'].values
    summry['top value'] = desc['top'].values
    summry['Freq'] = desc['freq'].values

    return summry

In [57]:
summary(df).style.set_caption("**Summary of the product Data**").\
background_gradient(cmap='Pastel2_r', axis=0). \
set_properties(**{'border': '1.3px dotted', 'color': '', 'caption-side': 'left'})

,data type,#missing,Duplicate,#unique,min,max,avg,std dev,top value,Freq
NO,int64,0,0,104488,1.000000,104488.000000,52244.500000,30163.231801,nan,nan
ROWNUM,object,0,0,104487,nan,nan,nan,nan,0:00,2
totalCount,object,0,0,4,nan,nan,nan,nan,104488,104451
pdt_cd,object,0,0,104488,nan,nan,nan,nan,100001,1
pdt_share,object,0,0,1,nan,nan,nan,nan,Y,104488
pdt_share_txt,object,0,0,1,nan,nan,nan,nan,공유,104488
send_chk,object,0,0,2,nan,nan,nan,nan,Y,104466
send_chk_txt,object,0,0,2,nan,nan,nan,nan,전송완료,104466
pdt_code,object,3,0,76,nan,nan,nan,nan,SZ,46466
pdt_name,object,1,0,101448,nan,nan,nan,nan,삭제,98


# 인사이트 
spec부분에 상세규격 별도기재 부분에 대한 해석을 논의해보아야함.     
104488개 중에 101448개의 이름이 일치하지 않음. < 분류가 필요한 이유     
pdt_name에 brand 이름이 들어간 경우를 처리해야하는데 어떻게 할지. 우선 괄호를 다 지우고 단어들을 토큰화 시킬지.

In [66]:
# 원본 데이터는 df
df_test = df.copy()

import re

def preprocess_pdt_name(name):
    name = str(name).lower()  # object → 문자열 + 소문자
    # 괄호, 역슬래시, 슬래시 제거하고 위치에 공백 한 칸 추가
    name = re.sub(r'[()/\\]', ' ', name)
    # 연속된 공백을 하나로 통일
    name = re.sub(r'\s+', ' ', name)
    return name.strip()  # 앞뒤 공백 제거

# df_test에 적용
df_test['pdt_name'] = df_test['pdt_name'].apply(preprocess_pdt_name)

# 확인
print(df_test[['pdt_name']].head(10))



                        pdt_name
0                 전산기록지 80컬럼 양미싱
1                전산기록지 132컬럼 양미싱
2              팩스용지 15mx210mm 한솔
3              팩스용지 15mx216mm 한솔
4              팩스용지 30mx210mm 한솔
5              팩스용지 30mx216mm 한솔
6     복사용지a4 80g 더블에이 500매x5권 박스
7  컬러레이저용지a4 90g 500매 neusiedler
8  컬러레이저용지a3 90g 500매 neusiedler
9     복사용지b4 80g 더블에이 500매x5권 박스


In [67]:
# 공백 기준 토큰화
df_test['tokens'] = df_test['pdt_name'].apply(lambda x: x.split())

# 확인
print(df_test[['pdt_name','tokens']].head(10))


                        pdt_name                              tokens
0                 전산기록지 80컬럼 양미싱                  [전산기록지, 80컬럼, 양미싱]
1                전산기록지 132컬럼 양미싱                 [전산기록지, 132컬럼, 양미싱]
2              팩스용지 15mx210mm 한솔               [팩스용지, 15mx210mm, 한솔]
3              팩스용지 15mx216mm 한솔               [팩스용지, 15mx216mm, 한솔]
4              팩스용지 30mx210mm 한솔               [팩스용지, 30mx210mm, 한솔]
5              팩스용지 30mx216mm 한솔               [팩스용지, 30mx216mm, 한솔]
6     복사용지a4 80g 더블에이 500매x5권 박스    [복사용지a4, 80g, 더블에이, 500매x5권, 박스]
7  컬러레이저용지a4 90g 500매 neusiedler  [컬러레이저용지a4, 90g, 500매, neusiedler]
8  컬러레이저용지a3 90g 500매 neusiedler  [컬러레이저용지a3, 90g, 500매, neusiedler]
9     복사용지b4 80g 더블에이 500매x5권 박스    [복사용지b4, 80g, 더블에이, 500매x5권, 박스]


In [69]:
# 공백 기준 토큰화
df_test['tokens'] = df_test['pdt_name'].apply(lambda x: x.split())

# 확인
print(df_test[['pdt_name','tokens']].head(10))


                        pdt_name                              tokens
0                 전산기록지 80컬럼 양미싱                  [전산기록지, 80컬럼, 양미싱]
1                전산기록지 132컬럼 양미싱                 [전산기록지, 132컬럼, 양미싱]
2              팩스용지 15mx210mm 한솔               [팩스용지, 15mx210mm, 한솔]
3              팩스용지 15mx216mm 한솔               [팩스용지, 15mx216mm, 한솔]
4              팩스용지 30mx210mm 한솔               [팩스용지, 30mx210mm, 한솔]
5              팩스용지 30mx216mm 한솔               [팩스용지, 30mx216mm, 한솔]
6     복사용지a4 80g 더블에이 500매x5권 박스    [복사용지a4, 80g, 더블에이, 500매x5권, 박스]
7  컬러레이저용지a4 90g 500매 neusiedler  [컬러레이저용지a4, 90g, 500매, neusiedler]
8  컬러레이저용지a3 90g 500매 neusiedler  [컬러레이저용지a3, 90g, 500매, neusiedler]
9     복사용지b4 80g 더블에이 500매x5권 박스    [복사용지b4, 80g, 더블에이, 500매x5권, 박스]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_test['tokens_str'])

# 코사인 유사도 계산
similarity = cosine_similarity(X)

# 임계값 기반 그룹화
threshold = 0.7
n = X.shape[0]
group_ids = np.full(n, -1)  # 초기 그룹 -1

current_group = 0
for i in range(n):
    if group_ids[i] == -1:
        group_ids[i] = current_group
        # i와 유사도가 threshold 이상인 다른 항목들을 같은 그룹으로
        similar_items = np.where(similarity[i] >= threshold)[0]
        group_ids[similar_items] = current_group
        current_group += 1

df_test['임의그룹'] = group_ids

# 확인
print(df_test[['pdt_name','임의그룹']].head(20))

